In [1]:
import gradio as gr
import ollama
from duckduckgo_search import DDGS

/opt/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
available_models = [model["model"] for model in ollama.list()["models"]]

In [3]:
ddgs = DDGS()
def search(query):
    print(f"Searching for: {query}")
    response = ""
    results = ddgs.text(query, max_results=10, region="in-en", safesearch="moderate")
    for r in results:
        response += r["body"]
    return response

In [4]:
system_message = "You are a helpful assistant that responds in markdown format."
system_message += "Give concise and accurate answers to the user's questions."
system_message += "Always be accuracte. If you don't know the anaswer, say 'I don't know'."
model = "llama3.2:latest"

In [5]:
search_function = {
    "name": "search",
    "description": "Searches the web for information.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to use."
            }
        },
        "required": ["query"]
    }
}

In [6]:
tools = [{"type": "function", "function": search_function}]

In [7]:
def handle_tool_call(message):
    tool_calls = message.get("tool_calls", [])
    for tool in tool_calls:
        if tool.get("function", {}).get("name") == "search":
            query = tool["function"]["arguments"].get("query", "")
            result = search(query)
    print(f"Tool call result: {result}")
    return {"role": "tool", "name": "search", "content": result}   

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})
    response = ollama.chat(model=model, messages=messages, tools=tools)

    if response["message"].get("tool_calls",[]):
        message = response["message"]
        tool_response = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        response = ollama.chat(model=model, messages=messages)
    return response["message"]["content"]


In [ ]:
gr.ChatInterface(fn=chat).launch()

/opt/miniconda3/envs/rag/lib/python3.12/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Searching for: IPL winner
Tool call result: Find out the winners, runners-up, venues, and player of the tournament of each Indian Premier League (IPL) season from 2008 to 2023. Chennai Super Kings and Mumbai Indians have won the most titles with five each, while Kolkata Knight Riders have won three.Find out the list of IPL winners and runners up of all seasons, along with the captain, man of the match and player of the series of each edition. Mumbai Indians and Chennai Super Kings are the most successful teams with five titles each.IPL 2025 final prize money: How much money do winner & runner-up win? RCB crowned IPL 2025 champion after 6-run victory over PBKS in the final. IPL 2010: Chennai Super Kings. MS Dhoni's Chennai Super Kings finally broke their title drought in 2010. They had a steady league phase, winning seven games to finish third. In the semi-final, they outplayed Deccan Chargers. The final was a high-pressure clash with Mumbai Indians, and CSK rose to the occasion, winnin